In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/523-Project/icpr2020dfdc/

/content/gdrive/MyDrive/523-Project/icpr2020dfdc


In [ ]:
!pip install efficientnet_pytorch  albumentations==0.4.6

In [ ]:
import torch
from torch.utils.model_zoo import load_url
from PIL import Image
import matplotlib.pyplot as plt

import sys
sys.path.append('..')

from blazeface import FaceExtractor, BlazeFace
from architectures import fornet,weights
from isplutils import utils

In [ ]:
"""
Choose an architecture between
- EfficientNetB4
- EfficientNetB4ST
- EfficientNetAutoAttB4
- EfficientNetAutoAttB4ST
- Xception
"""
net_model = 'EfficientNetB4'

"""
Choose a training dataset between
- DFDC
- FFPP
"""
train_db = 'DFDC'

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
face_policy = 'scale'
face_size = 224

In [ ]:
model_url = weights.weight_url['{:s}_{:s}'.format(net_model,train_db)]
net = getattr(fornet,net_model)().eval().to(device)
net.load_state_dict(load_url(model_url,map_location=device,check_hash=True))

In [ ]:
transf = utils.get_transformer(face_policy, face_size, net.get_normalizer(), train=False)

In [ ]:
facedet = BlazeFace().to(device)
facedet.load_weights("./blazeface/blazeface.pth")
facedet.load_anchors("./blazeface/anchors.npy")
face_extractor = FaceExtractor(facedet=facedet)

In [ ]:
def predictor(im):
  im = face_extractor.process_image(img=im)
  
  if(len(im['faces'])) == 0:
    return 0
  im = im['faces'][0] # take the face with the highest confidence score found by BlazeFace   

  faces_t = torch.stack( [ transf(image=im)['image']] )

  with torch.no_grad():
    faces_pred = torch.sigmoid(net(faces_t.to(device))).cpu().numpy().flatten()  
  return faces_pred[0]

In [ ]:
GLOBAL_PATH = '/content/gdrive/MyDrive/523-Project/FakeAVCeleb_v1.2/FakeVideo-RealAudio/'
#write_path = '/content/gdrive/MyDrive/523-Project/FakeAVCeleb_v1.2/'
import os
import cv2
import numpy as np

with open(GLOBAL_PATH + 'preds.txt' , 'a') as f:
  for race in os.listdir(GLOBAL_PATH):
    if race == 'preds.txt':
      continue
    temp1 = GLOBAL_PATH+race+'/'
    for gender in os.listdir(temp1):
      temp2 = temp1 + gender + '/'
      for id in os.listdir(temp2):
        temp3 = temp2 + id +'/'
        for file in os.listdir(temp3):
          if file.endswith('.txt'):
            continue
          temp4 = temp3 + file
          vidcap = cv2.VideoCapture(temp4)
          vals = np.zeros(30)
          for i in range(0,30):
            avg = 0.0
            exi,frame = vidcap.read()
            if exi == True:
              vals[i] = predictor(frame)
            else:
              break
          if i != 0:
            f.write(temp4  + '  ' + net_model + '  ' +str(np.mean(vals))  + '   -->  ' + str(1) + '\n')
            f.flush()
          else:
            continue  


